# Исследование надежности заемщиков.

Нужно разобраться, как влияют семейное положение, количество детей, уровень дохода и цели кредита на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Результаты исследования будут учтены при построении модели кредитного скоринга.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd

df = pd.read_csv('/datasets/data.csv')
display(df.head())
df.info()
display(df. describe())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


В столбце `days_employed` сразу видно аномальные значения - минусовые и очень большие.
Названия столбцов соответствуют стилю.

В `describe` сразу видно аномальные `min` и `max` в категориях `children`, `days_employed`, `dob_years`.

### Шаг 2.1 Заполнение пропусков


In [2]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски есть в двух столбцах и одинаковое количество. 

In [3]:
display(df[df['days_employed'].isna()].head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Посчитаем процент пропусков.

In [4]:
print(f'Процент пропусков: {df.days_employed.isna().sum() / df.shape[0]:.0%}')

Процент пропусков: 10%


Проверим на отрицательные значения столбец `total_income`.

In [5]:
df[df['total_income'] < 0].value_counts()

Series([], dtype: int64)

В колонке `days_employed` есть аномальные значения, пропуски заменим после исправления.
В колонке `total_income` заменим пропуски на медиану. Среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства. По этому берем медиану.

In [6]:
df['total_income'] = df['total_income'].fillna(value=df['total_income'].median())
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Одинаковое кол-во пропусков приходится на `days_employed` и `income_type`, возможно они как то связаны и произошла ошибка при выгрузке. Возможно это закрытая информация этих клиентов. Нужно уточнить у тех, кто предоставил данные.

### Шаг 2.2 Проверка данных на аномалии и исправления.

Проверим на отрицательные значения `days_employed` и исправим. Очень похоже на ошибку, нужно привести значения к абсолютным.

In [7]:
display(df[df['days_employed'] < 0].head())
df[df['days_employed'] < 0][['days_employed']].count()

df.loc[df['days_employed'] < 0, 'days_employed'] = abs(df.loc[df['days_employed'] < 0, 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья


Большие цифры в колонке `days_employed` возможно записаны не в количестве дней, а количестве часов. Проверим стаж и соотнесем значения с возрастом. Возьмем все значения где стаж больше возраста.

In [8]:
display(df[df['days_employed'] > 360 * df['dob_years']].head())

test_df = df[df['days_employed'] > 360 * df['dob_years']].head(10)
display(test_df['days_employed'] / 24 / 360, test_df['dob_years'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью


4     39.382647
18    46.328835
24    39.184254
25    42.077371
30    38.840471
35    45.604291
50    40.941138
56    42.840867
71    39.133510
78    41.634600
Name: days_employed, dtype: float64

4     53
18    53
24    57
25    67
30    62
35    68
50    63
56    64
71    62
78    61
Name: dob_years, dtype: int64

Не совсем сходится, для 53-х летнего стаж 46 лет. Гипотеза не подтвердилась, но зато можно заметить, что большие значения в группе "пенсионер". Проверим, если так и есть, то заменим аномальные значения медианой для группы `income_type = пенсионер`

In [9]:
print(df[df['days_employed'] > 360 * df['dob_years']].groupby('income_type')['days_employed'].count())

income_type
безработный       2
госслужащий       6
компаньон        18
пенсионер      3443
сотрудник        50
Name: days_employed, dtype: int64


При детальном анализе выяснилось, что у всей группы `пенсионер` аномально большие значения. По хорошему нужно узнать, что это за цифры, возможно не правильный расчет при выгрузке или не привели значения к "годам".
Т.к. нет возможности, то будем заменять все аномально большие значения на медиану по всему столбцу `days_employed`.

In [10]:
df.loc[df['days_employed'] > 360 * df['dob_years'], 'days_employed'] = df['days_employed'].median()

Так же заменим пропуски на медиану.

In [11]:
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Поищем еще аномалии в данных

In [12]:
print(df['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Явные аномалии `-1` и `20`, возникшие при некорректном вводе данных. Заменим на `1` и `2`

In [13]:
df.loc[df['children'] == -1, 'children'] = 1
df.loc[df['children'] == 20, 'children'] = 2
print(df['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


In [14]:
print(df['dob_years'].value_counts())

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


Возраст "0" явная аномалия, заменим ее на среднее значение. Но можно посмотреть к каким категориям они относятся и заменить возраст на среднее значение по этой категории.

In [15]:
print(df[df['dob_years'] == 0].groupby('income_type')['dob_years'].count())

income_type
госслужащий     6
компаньон      20
пенсионер      20
сотрудник      55
Name: dob_years, dtype: int64


In [16]:
df.loc[df['income_type'] == 'госслужащий', 'dob_years'] = int(df[df['income_type'] == 'госслужащий']['dob_years'].mean())
df.loc[df['income_type'] == 'компаньон', 'dob_years'] = int(df[df['income_type'] == 'компаньон']['dob_years'].mean())
df.loc[df['income_type'] == 'пенсионер', 'dob_years'] = int(df[df['income_type'] == 'пенсионер']['dob_years'].mean())
df.loc[df['income_type'] == 'сотрудник', 'dob_years'] = int(df[df['income_type'] == 'сотрудник']['dob_years'].mean())

Проверим отсутствие нулей в возрасте.

In [17]:
print(df[df['dob_years'] == 0].groupby('income_type')['dob_years'].count())

Series([], Name: dob_years, dtype: int64)


### Шаг 2.3. Изменение типов данных.

Изменим тип данных `days_employed` и `total_income` на целочисленный.

In [18]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Найдем и удалим явные дубликаты

In [19]:
df.duplicated().sum()

670

In [20]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

Проверим колонки на не явные дубликаты

In [21]:
print(df['education'].value_counts())

среднее                13193
высшее                  4614
СРЕДНЕЕ                  770
Среднее                  707
неоконченное высшее      667
ВЫСШЕЕ                   273
Высшее                   267
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


Приведем к единому стилю/регистру.

In [22]:
df['education'] = df['education'].str.lower()

In [23]:
print(df['education'].value_counts())

среднее                14670
высшее                  5154
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64


In [24]:
print(df['family_status'].value_counts())

женат / замужем          11928
гражданский брак          4032
Не женат / не замужем     2765
в разводе                 1185
вдовец / вдова             945
Name: family_status, dtype: int64


Дубликатов нет, просто приведем все к единому регистру.

In [25]:
df['family_status'] = df['family_status'].str.lower()

In [26]:
print(df['family_status'].value_counts())

женат / замужем          11928
гражданский брак          4032
не женат / не замужем     2765
в разводе                 1185
вдовец / вдова             945
Name: family_status, dtype: int64


In [27]:
print(df['gender'].value_counts())

F      13740
M       7114
XNA        1
Name: gender, dtype: int64


Всего одна строка с неверным значением, можно удалить или присвоить значение "F" или "M". Одна строка не повлияет на статистику. Присвоим значение "M".

In [28]:
df.loc[df['gender'] == 'XNA', 'gender'] = 'M'
print(df['gender'].value_counts())

F    13740
M     7115
Name: gender, dtype: int64


In [29]:
print(df['income_type'].value_counts())

сотрудник          10730
компаньон           4971
пенсионер           3701
госслужащий         1447
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64


Проверим не появились ли явные дубликаты, после исправления неявных. Если появились - удалим.

In [30]:
print(df.duplicated().sum())

98


In [31]:
df = df.drop_duplicates().reset_index(drop=True)
print(df.duplicated().sum())

0


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20757 entries, 0 to 20756
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          20757 non-null  int64 
 1   days_employed     20757 non-null  int64 
 2   dob_years         20757 non-null  int64 
 3   education         20757 non-null  object
 4   education_id      20757 non-null  int64 
 5   family_status     20757 non-null  object
 6   family_status_id  20757 non-null  int64 
 7   gender            20757 non-null  object
 8   income_type       20757 non-null  object
 9   debt              20757 non-null  int64 
 10  total_income      20757 non-null  int64 
 11  purpose           20757 non-null  object
dtypes: int64(7), object(5)
memory usage: 1.9+ MB


In [33]:
education_dict = df[['education_id', 'education']]
print(education_dict.head(10))

   education_id education
0             0    высшее
1             1   среднее
2             1   среднее
3             1   среднее
4             1   среднее
5             0    высшее
6             0    высшее
7             1   среднее
8             0    высшее
9             1   среднее


In [34]:
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
print(education_dict)

   education_id            education
0             0               высшее
1             1              среднее
2             2  неоконченное высшее
3             3            начальное
4             4       ученая степень


In [35]:
family_status_dict = df[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
print(family_status_dict)

   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 2         вдовец / вдова
3                 3              в разводе
4                 4  не женат / не замужем


Удалим не нужные столбцы.

In [36]:
df.drop(columns=['education', 'family_status'], axis=1, inplace=True)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20757 entries, 0 to 20756
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          20757 non-null  int64 
 1   days_employed     20757 non-null  int64 
 2   dob_years         20757 non-null  int64 
 3   education_id      20757 non-null  int64 
 4   family_status_id  20757 non-null  int64 
 5   gender            20757 non-null  object
 6   income_type       20757 non-null  object
 7   debt              20757 non-null  int64 
 8   total_income      20757 non-null  int64 
 9   purpose           20757 non-null  object
dtypes: int64(7), object(3)
memory usage: 1.6+ MB


### Шаг 2.6. Категоризация дохода.

In [38]:
def income_category(income):
    """
    Возвращает категорию дохода:
    0–30000 — 'E';
    30001–50000 — 'D';
    50001–200000 — 'C';
    200001–1000000 — 'B';
    1000001 и выше — 'A'.
    """
    if income <= 30000:
        return 'E'
    if income <= 50000 and income >= 30001:
        return 'D'
    if income <= 200000 and income >= 50001:
        return 'C'
    if income <= 1000000 and income >= 200001:
        return 'B'
    else:
        return 'A'

In [39]:
df['total_income_category'] = df['total_income'].apply(income_category)
display(df.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,39,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,39,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,39,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,39,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,2194,59,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

Изучим данные в столбце `purpose`

In [40]:
print(df.purpose.value_counts())

свадьба                                   755
сыграть свадьбу                           722
на проведение свадьбы                     720
операции с недвижимостью                  658
покупка коммерческой недвижимости         640
операции с коммерческой недвижимостью     633
покупка жилья                             632
операции с жильем                         631
покупка жилья для сдачи                   626
жилье                                     625
покупка жилья для семьи                   617
строительство собственной недвижимости    610
строительство жилой недвижимости          608
недвижимость                              608
покупка своего жилья                      606
операции со своей недвижимостью           602
строительство недвижимости                600
покупка недвижимости                      594
ремонт жилью                              589
покупка жилой недвижимости                581
на покупку своего автомобиля              486
автомобиль                        

In [41]:
def purpose_category(purpose):
    """
    Возвращает одну из категорий, на основе совпадений:
    'операции с автомобилем',
    'операции с недвижимостью',
    'проведение свадьбы',
    'получение образования'
    """
    if 'автом' in purpose:
        return 'операции с автомобилем'
    if 'жиль' in purpose or 'недвиж' in purpose:
        return 'операции с недвижимостью'
    if 'свадь' in purpose:
        return 'проведение свадьбы'
    if 'образ' in purpose:
        return 'получение образования'
    else:
        return 'unknow'

Проверим все ли строки распределились по группам, и нет ли `unknow`

In [42]:
df['purpose_category'] = df['purpose'].apply(purpose_category)
print(df['purpose_category'].value_counts())

операции с недвижимостью    10460
операции с автомобилем       4189
получение образования        3911
проведение свадьбы           2197
Name: purpose_category, dtype: int64


### Ответы на вопросы.

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?:

In [43]:
def calc_debt_percent(column):
    """
    Функция позволяет расчитать процент просрочек для определенной группы (столбца).
    Возвращает сводную таблицу.
    """
    pivot = df.pivot_table(index=column,  values='debt', aggfunc=['count', 'sum'])
    pivot['percent'] = round(pivot['count'] / pivot['sum'], 2)
    return pivot.droplevel(level=1, axis=1)

In [44]:
display(calc_debt_percent('children'))

,count,sum,percent
children,,,
0,13551,1051,12.89
1,4737,444,10.67
2,2090,202,10.35
3,329,27,12.19
4,41,4,10.25
5,9,0,inf


Зависимость есть, но совсем не большая - без детей и с 3 детьми процент просрочек немного больше.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?:

In [45]:
display(calc_debt_percent('family_status_id'), family_status_dict)

,count,sum,percent
family_status_id,,,
0,11869,925,12.83
1,4005,381,10.51
2,940,63,14.92
3,1182,85,13.91
4,2761,274,10.08


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


Зависимость есть, не в браке "без обязательств" меньше всего просрочек, в разводе и в браке уже больше и больше всего просрочек у группы `вдовец / вдова`

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [46]:
display(calc_debt_percent('total_income_category'))

,count,sum,percent
total_income_category,,,
A,25,2,12.50
B,5041,356,14.16
C,15319,1347,11.37
D,350,21,16.67
E,22,2,11.00


Да, тоже есть небольшая зависимость, но на категорию `D` маленькая выборка, а на `А` и `Е` вообще минимальная.

#### Как разные цели кредита влияют на его возврат в срок?

In [47]:
display(calc_debt_percent('purpose_category'))

,count,sum,percent
purpose_category,,,
операции с автомобилем,4189,401,10.45
операции с недвижимостью,10460,780,13.41
получение образования,3911,368,10.63
проведение свадьбы,2197,179,12.27


Просрочек немного больше у групп - `операции с недвижимостью` и `проведение свадьбы`

## Общий вывод:

При анализе и предобработке данных были найдены и исправлены небольшие аномалии в данных. Стоит обратить внимание на большое количество неверных данных в зарплатах пенсионеров и пропуски стажа и зарплаты (уточнить данные).

Обработали и удалили явные и неявные дубликаты. Категориальные данные были приведены к единому стилю. 
Сделана категоризация данных - цели кредита были обработаны и распределены по категориям, сделано разделение по категориям дохода.

Сделали декомпозицию исходного датафрейма, сформировали дополнительные датафреймы словарей.

Провели анализ данных как разные категории влияют на количество просрочек. Зависимости есть, если накладывать их на конкретного человека, то можно составить его кредитный рейтинг относительно других.